In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install -q transformers
!pip install -q datasets

In [45]:
!pip install -U bitsandbytes

In [3]:
from transformers import AutoTokenizer
from datasets import load_dataset

In [8]:
template_tokenizer = AutoTokenizer.from_pretrained(
    'TinyLlama/TinyLlama-1.1B-Chat-v1.0'
)

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

In [19]:
def format_prompt(example):
    chat = example['messages']
    prompt = template_tokenizer.apply_chat_template(chat, tokenize=False)
    return {"text": prompt}

In [21]:
dataset = (
    load_dataset('HuggingFaceH4/ultrachat_200k', split='test_sft')
        .shuffle(seed=42)
        .select(range(3_000))
)

README.md:   0%|          | 0.00/3.90k [00:00<?, ?B/s]

(…)-00000-of-00003-a3ecf92756993583.parquet:   0%|          | 0.00/244M [00:00<?, ?B/s]

(…)-00001-of-00003-0a1804bcb6ae68c6.parquet:   0%|          | 0.00/244M [00:00<?, ?B/s]

(…)-00002-of-00003-ee46ed25cfae92c6.parquet:   0%|          | 0.00/244M [00:00<?, ?B/s]

(…)-00000-of-00001-f7dfac4afe5b93f4.parquet:   0%|          | 0.00/81.2M [00:00<?, ?B/s]

...

(…)-00000-of-00001-3d4cd8309148a71f.parquet:   0%|          | 0.00/80.4M [00:00<?, ?B/s]

Generating train_sft split:   0%|          | 0/207865 [00:00<?, ? examples/s]

Generating test_sft split:   0%|          | 0/23110 [00:00<?, ? examples/s]

Generating train_gen split:   0%|          | 0/256032 [00:00<?, ? examples/s]

Generating test_gen split:   0%|          | 0/28304 [00:00<?, ? examples/s]

In [23]:
print(dataset)

Dataset({
    features: ['prompt', 'prompt_id', 'messages'],
    num_rows: 3000
})


In [29]:
dataset = dataset.map(format_prompt)

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [34]:
dataset['text'][2576]

'<|user|>\nGiven the text: Knock, knock. Who’s there? Hike.\nCan you continue the joke based on the given text material "Knock, knock. Who’s there? Hike"?</s>\n<|assistant|>\nSure! Knock, knock. Who\'s there? Hike. Hike who? Hike up your pants, it\'s cold outside!</s>\n<|user|>\nCan you tell me another knock-knock joke based on the same text material "Knock, knock. Who\'s there? Hike"?</s>\n<|assistant|>\nOf course! Knock, knock. Who\'s there? Hike. Hike who? Hike your way over here and let\'s go for a walk!</s>\n'

In [36]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

In [37]:
model_name = 'TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T'

In [40]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype='float16',
    bnb_4bit_use_double_quant=True
)

In [48]:
!pip uninstall bitsandbytes -y
!pip install bitsandbytes --no-cache-dir

Found existing installation: bitsandbytes 0.45.3
Uninstalling bitsandbytes-0.45.3:
  Successfully uninstalled bitsandbytes-0.45.3
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 272.3 MB/s eta 0:00:00a 0:00:01


In [49]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map='auto',
    quantization_config=bnb_config
)

ImportError: Using `bitsandbytes` 4-bit quantization requires the latest version of bitsandbytes: `pip install -U bitsandbytes`

In [51]:
import bitsandbytes
print(bitsandbytes.__version__)

0.45.3


In [53]:
!pip index versions bitsandbytes

bitsandbytes (0.45.3)
Available versions: 0.45.3, 0.45.2, 0.45.1, 0.45.0, 0.44.1, 0.44.0, 0.43.3, 0.43.2, 0.43.1, 0.43.0, 0.42.0, 0.41.3.post2, 0.41.3.post1, 0.41.3, 0.41.2.post2, 0.41.2.post1, 0.41.2, 0.41.1, 0.41.0, 0.40.2, 0.40.1.post1, 0.40.1, 0.40.0.post4, 0.40.0.post3, 0.40.0.post2, 0.40.0.post1, 0.40.0, 0.39.1, 0.39.0, 0.38.1, 0.38.0.post2, 0.38.0.post1, 0.38.0, 0.37.2, 0.37.1, 0.37.0, 0.36.0.post2, 0.36.0.post1, 0.36.0, 0.35.4, 0.35.3, 0.35.2, 0.35.1, 0.35.0, 0.34.0, 0.33.1, 0.33.0, 0.32.3, 0.32.2, 0.32.1, 0.32.0, 0.31.8
  INSTALLED: 0.45.3
  LATEST:    0.45.3


In [54]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [ ]:
model.config.use_cache = False
model.config-